In [1]:
from conv_model import ConvProtein
from vocab import Vocab

c:\ProgramData\Miniconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = ConvProtein([21,32],[32,64,128,256,512])
model.load('./ckpt.21-32-32_64_128_256_512.pkl')

loaded model from ./ckpt.21-32-32_64_128_256_512.pkl
